# **One: Data Prepossesing and importing libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/analysis


In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['animation.embed_limit'] = 100
import numpy as np
import seaborn as sns
# Standard library imports
import time
import random
import heapq
from collections import defaultdict
import os
import sys

# Video creation imports
import imageio
from matplotlib import animation
import matplotlib.animation as animation

# Interactive visualization imports
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import HTML
!apt-get install -y ffmpeg

In [ ]:
def load_and_preprocess_network(filename):#function to read a graph from a file
  G=nx.read_edgelist(filename, data=(("weight", float),))
  return G

# **Two: Data Structure**

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):#sets root of the node's current parent as it's parent to avoid increasing the depth of the tree
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        return True

# **Three: Algorithms**

In [ ]:

def kruskal_mst_incremental(G):
    n = G.number_of_nodes()#gets the number of nodes to know when to break out of the loop
    node_map = {node: i for i, node in enumerate(G.nodes())}#maps each node to a unique integer as it is used later on as integer indicies
    uf = UnionFind(n)#initalize a disjoint set union with n elements the algroithm uses it later on to check for cycles

    # Edge format: (u, v, weight) extracts all edges
    edges = [(u, v, G[u][v]['weight']) for u, v in G.edges()]#this format is needed to sort them and process them in ascending order
    edges.sort(key=lambda x: x[2])#sorts the edges based on their weights

    mst_edges = []

    for u, v, weight in edges:#loops over the edges
        u_idx, v_idx = node_map[u], node_map[v]#retrieves the unique integer for each node
        if uf.union(u_idx, v_idx):#checks if these two nodes are in disjoint components (checks for a cycle)
            mst_edges.append((u, v, weight))
            if len(mst_edges) == n - 1:
                break

    return mst_edges

In [ ]:
def prim_mst_incremental(G):
    start_node = next(iter(G.nodes()))
    mst_edges = []
    visited = {start_node}

    # Priority queue: (weight, node1, node2)
    heap = []
    for neighbor in G.neighbors(start_node):
        heapq.heappush(heap, (G[start_node][neighbor]['weight'], start_node, neighbor))

    while heap and len(visited) < G.number_of_nodes():
        weight, u, v = heapq.heappop(heap)

        if v in visited:
            continue

        visited.add(v)
        mst_edges.append((u, v, weight))

        for neighbor in G.neighbors(v):
            if neighbor not in visited:
                heapq.heappush(heap, (G[v][neighbor]['weight'], v, neighbor))

    return mst_edges

In [ ]:
import heapq

def prim_mst_incremental_using_adj(adj_list):
    visited = set()
    mst_edges = []

    start_node = list(adj_list.keys())[0]
    visited.add(start_node)
    heap = []

    for neighbor, weight in adj_list[start_node]:
        heapq.heappush(heap, (weight, start_node, neighbor))

    while heap and len(visited) < len(adj_list):
        weight, u, v = heapq.heappop(heap)
        if v in visited:
            continue
        visited.add(v)
        mst_edges.append((u, v, weight))
        for neighbor, w in adj_list[v]:
            if neighbor not in visited:
                heapq.heappush(heap, (w, v, neighbor))

    return mst_edges

In [ ]:
def boruvka_mst_incremental(G):
    n = G.number_of_nodes()
    node_map = {node: i for i, node in enumerate(G.nodes())}
    uf = UnionFind(n)

    mst_edges = []

    while len(mst_edges) < n - 1:
        cheapest = [-1] * n

        for u, v in G.edges():
            weight = G[u][v]['weight']
            u_idx, v_idx = node_map[u], node_map[v]
            comp_u, comp_v = uf.find(u_idx), uf.find(v_idx)

            if comp_u != comp_v:
                if cheapest[comp_u] == -1 or weight < cheapest[comp_u][0]:
                    cheapest[comp_u] = (weight, u, v)
                if cheapest[comp_v] == -1 or weight < cheapest[comp_v][0]:
                    cheapest[comp_v] = (weight, u, v)

        for edge_info in cheapest:
            if edge_info != -1:
                weight, u, v = edge_info
                u_idx, v_idx = node_map[u], node_map[v]
                if uf.union(u_idx, v_idx):
                    mst_edges.append((u, v, weight))

    return mst_edges

In [ ]:
def reverse_delete_mst_incremental(G):
    edges = [(G[u][v]['weight'], u, v) for u, v in G.edges()]
    edges.sort(reverse=True)  # Start with heaviest edges

    current_graph = G.copy()
    removed_edges = []

    for weight, u, v in edges:
        current_graph.remove_edge(u, v)#removes the edge
        if not nx.is_connected(current_graph):#checks if the graph becomess disconnected
            current_graph.add_edge(u, v, weight=weight)#if true then it can't remove this edge and adds it back to the graph
        else:
            removed_edges.append((u, v, weight))#else the edge it removed and added to the list of removed edges

    mst_edges = [(u, v, current_graph[u][v]['weight']) for u, v in current_graph.edges()]#the remaining edges are the ones which form the mst
    return mst_edges #to return the mst of the graph

In [ ]:
import random

def karger_modified_mst(G, iterations=100):
    best_mst = []
    best_cost = float('inf')
    nodes = list(G.nodes())

    for _ in range(iterations):
        edges = list(G.edges(data=True))
        random.shuffle(edges)

        uf = UnionFind(len(nodes))
        node_index = {node: i for i, node in enumerate(nodes)}
        mst = []
        cost = 0

        for u, v, data in edges:
            idx_u, idx_v = node_index[u], node_index[v]
            if uf.union(idx_u, idx_v):
                weight = data['weight']
                mst.append((u, v, weight))
                cost += weight
                if len(mst) == len(nodes) - 1:
                    break

        if len(mst) == len(nodes) - 1 and cost < best_cost:
            best_cost = cost
            best_mst = mst

    return best_mst

# **Four: Animation Functions**

In [ ]:
from google.colab import files

def visualize_mst_animation_kruskal(G_full, step=1, layout_func=nx.spring_layout, interval=1000, output_file="mst_animation.mp4"):
    # Load the full graph
    pos = layout_func(G_full, k=1.0)

    # Kruskal's MST + timing
    start = time.time()
    mst_edges = kruskal_mst_incremental(G_full)
    end = time.time()
    cost = sum(weight for u, v, weight in mst_edges)
    round_cost=round(cost,2)
    excution_time=round(end-start,4)


    # Show metrics
    print("✅ MST Total Cost: ",round_cost)
    print("⏱️ Execution Time: ",excution_time," seconds")
    print("🎥 Saving animation to: ",output_file)

    # Prepare animation graph
    fig, ax = plt.subplots(figsize=(6, 5))

    def init():
        ax.clear()
        nx.draw(G_full, pos, ax=ax, with_labels=True, node_color='lightgray', edge_color='lightgray')
        ax.set_title("Initial Graph")
        return ax,

    def update(frame):
        ax.clear()
        G = nx.Graph()
        step_size = step
        end_idx = min((frame + 1) * step_size, len(mst_edges))
        edges_to_show = mst_edges[:end_idx]
        G.add_weighted_edges_from(edges_to_show)
        cost1 = sum(weight for u,v, weight in edges_to_show)
        nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightgreen', edge_color='blue', width=2)
        ax.set_title(f"Step {frame+1}, showing {len(edges_to_show)} edges, current cost: {cost1}")
        return ax,

    num_steps = (len(mst_edges) + step- 1) // step  # Ceiling division
    ani = animation.FuncAnimation(
        fig, update, frames=num_steps, init_func=init, interval=interval,
        blit=False, repeat=False
    )

    # Save video
    ani.save(output_file, writer='ffmpeg', fps=1000//interval)
    print("✅ Animation saved successfully.")
    # Also return interactive animation in notebook
    plt.close()
    return  HTML(ani.to_jshtml())

In [ ]:
def visualize_mst_animation_prim_mst(G_full, step=1,layout_func=nx.spring_layout, interval=1000, output_file="mst_animation.mp4"):
    # Load the full graph
    pos = layout_func(G_full, k=1.0)

    # Kruskal's MST + timing
    start = time.time()
    mst_edges = prim_mst_incremental(G_full)
    end = time.time()
    cost = sum(weight for u,v, weight in mst_edges)
    round_cost=round(cost,2)
    excution_time=round(end-start,4)

    # Show metrics
    print("✅ MST Total Cost: ",round_cost)
    print("⏱️ Execution Time: ",excution_time," seconds")
    print("🎥 Saving animation to: ", output_file)

    # Prepare animation graph
    fig, ax = plt.subplots(figsize=(6, 5))
    cost=0
    def init():
        ax.clear()
        nx.draw(G_full, pos, ax=ax, with_labels=True, node_color='lightgray', edge_color='lightgray')
        ax.set_title("Initial Graph")
        return ax,

    def update(frame):
        ax.clear()
        G = nx.Graph()
        step_size = step
        end_idx = min((frame + 1) * step_size, len(mst_edges))
        edges_to_show = mst_edges[:end_idx]
        G.add_weighted_edges_from(edges_to_show)
        cost1 = sum(weight for u,v, weight in edges_to_show)
        nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightgreen', edge_color='blue', width=2)
        ax.set_title(f"Step {frame+1}, showing {len(edges_to_show)} edges, current cost: {cost1}")
        return ax,

    num_steps = (len(mst_edges) + step- 1) // step  # Ceiling division
    ani = animation.FuncAnimation(
        fig, update, frames=num_steps, init_func=init, interval=interval,
        blit=False, repeat=False
    )

    # Save video
    ani.save(output_file, writer='ffmpeg', fps=1000//interval)
    print("✅ Animation saved successfully.")

    # Also return interactive animation in notebook
    plt.close()
    return  HTML(ani.to_jshtml())

In [ ]:
def visualize_mst_animation_prim_mst_using_adj(G_full, step=1, layout_func=nx.spring_layout, interval=1000, output_file="prim_mst_animation.mp4"):
    # Load full graph
    pos = layout_func(G_full, k=1.0)

    # Optional: build adjacency list (for your own analysis if needed)
    adj_list = {
        node: [(nbr, G_full[node][nbr]['weight']) for nbr in G_full.neighbors(node)]
        for node in G_full.nodes()
    }

    # Run Prim's MST algorithm and time it
    start = time.time()
    mst_edges = prim_mst_incremental_using_adj(adj_list)
    end = time.time()
    cost = sum(weight for u, v, weight in mst_edges)
    cost=round(cost,2)
    excution_time=round(end-start,4)


    # Show metrics
    print("✅ MST Total Cost: ",cost)
    print("⏱️ Execution Time: ",excution_time," seconds")
    print("🎥 Saving animation to: ",output_file)

    # Prepare for animation

    fig, ax = plt.subplots(figsize=(6, 5))

    def init():
        ax.clear()
        nx.draw(G_full, pos, ax=ax, with_labels=True, node_color='lightgray', edge_color='lightgray')
        ax.set_title("Initial Graph")
        return ax,

    def update(frame):
        ax.clear()
        G = nx.Graph()
        step_size = step
        end_idx = min((frame + 1) * step_size, len(mst_edges))
        edges_to_show = mst_edges[:end_idx]
        G.add_weighted_edges_from(edges_to_show)
        cost1 = sum(weight for u,v, weight in edges_to_show)
        nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightgreen', edge_color='blue', width=2)
        ax.set_title(f"Step {frame+1}, showing {len(edges_to_show)} edges, current cost: {cost1}")
        return ax,

    num_steps = (len(mst_edges) + step- 1) // step  # Ceiling division
    ani = animation.FuncAnimation(
        fig, update, frames=num_steps, init_func=init, interval=interval,
        blit=False, repeat=False
    )


    ani.save(output_file, writer='ffmpeg', fps=1000//interval)
    print("✅ Animation saved successfully.")

    plt.close()
    return  HTML(ani.to_jshtml())

In [ ]:
def visualize_mst_animation_boruvka_mst(G_full, step=1, layout_func=nx.spring_layout, interval=1000, output_file="mst_animation.mp4"):
    # Load the full graph
    pos = layout_func(G_full, k=1.0)

    # Kruskal's MST + timing
    start = time.time()
    mst_edges = boruvka_mst_incremental(G_full)
    end = time.time()
    cost = sum(weight for u, v, weight in mst_edges)
    cost=round(cost,2)
    excution_time=round(end-start,4)

    # Show metrics
    print("✅ MST Total Cost: ",cost)
    print("⏱️ Execution Time: ",excution_time," seconds")
    print("🎥 Saving animation to: ",output_file)

    # Prepare animation graph

    fig, ax = plt.subplots(figsize=(6, 5))

    def init():
        ax.clear()
        nx.draw(G_full, pos, ax=ax, with_labels=True, node_color='lightgray', edge_color='lightgray')
        ax.set_title("Initial Graph")
        return ax,

    def update(frame):
        ax.clear()
        G = nx.Graph()
        step_size = step
        end_idx = min((frame + 1) * step_size, len(mst_edges))
        edges_to_show = mst_edges[:end_idx]
        G.add_weighted_edges_from(edges_to_show)
        cost1 = sum(weight for u,v, weight in edges_to_show)
        nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightgreen', edge_color='blue', width=2)
        ax.set_title(f"Step {frame+1}, showing {len(edges_to_show)} edges, current cost: {cost1}")
        return ax,

    num_steps = (len(mst_edges) + step- 1) // step  # Ceiling division
    ani = animation.FuncAnimation(
        fig, update, frames=num_steps, init_func=init, interval=interval,
        blit=False, repeat=False
    )


    # Save video
    ani.save(output_file, writer='ffmpeg', fps=1000//interval)
    print("✅ Animation saved successfully.")

    # Also return interactive animation in notebook
    plt.close()
    return  HTML(ani.to_jshtml())

In [ ]:
def visualize_mst_animation_reverse_delete_mst(G_full, step=1, layout_func=nx.spring_layout, interval=1000, output_file="mst_animation.mp4"):
    # Load the full graph
    pos = layout_func(G_full, k=1.0)

    # reverse's MST + timing
    start = time.time()
    mst_edges = reverse_delete_mst_incremental(G_full)
    end = time.time()
    cost = sum(weight for u, v, weight in mst_edges)
    cost=round(cost,2)
    excution_time=round(end-start,4)

    # Show metrics
    print("✅ MST Total Cost: ",cost)
    print("⏱️ Execution Time: ",excution_time," seconds")
    print("🎥 Saving animation to: ",output_file)

    # Prepare animation graph

    fig, ax = plt.subplots(figsize=(6, 5))

    def init():
        ax.clear()
        nx.draw(G_full, pos, ax=ax, with_labels=True, node_color='lightgray', edge_color='lightgray')
        ax.set_title("Initial Graph")
        return ax,

    def update(frame):
        ax.clear()
        G = nx.Graph()
        step_size = step
        end_idx = min((frame + 1) * step_size, len(mst_edges))
        edges_to_show = mst_edges[:end_idx]
        G.add_weighted_edges_from(edges_to_show)
        cost1 = sum(weight for u,v, weight in edges_to_show)
        nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightgreen', edge_color='blue', width=2)
        ax.set_title(f"Step {frame+1}, showing {len(edges_to_show)} edges, current cost: {cost1}")
        return ax,

    num_steps = (len(mst_edges) + step- 1) // step  # Ceiling division
    ani = animation.FuncAnimation(
        fig, update, frames=num_steps, init_func=init, interval=interval,
        blit=False, repeat=False
    )

    # Save video
    ani.save(output_file, writer='ffmpeg', fps=1000//interval)
    print("✅ Animation saved successfully.")

    # Also return interactive animation in notebook
    plt.close()
    return  HTML(ani.to_jshtml())

In [ ]:


def visualize_mst_animation_karger_modified_mst(G_full, step=1, layout_func=nx.spring_layout, interval=1000, output_file="mst_animation.mp4"):
    # Load graph
    pos = layout_func(G_full, k=1.0)

    # Call your MST function
    start = time.time()
    mst_edges = karger_modified_mst(G_full)  # Returns (u, v, {'weight': x})
    end = time.time()


    if not mst_edges:
        print("No MST found.")
        return

    # Compute total cost
    cost =sum(weight for u, v, weight in mst_edges)
    cost=round(cost,2)
    excution_time=round(end-start,4)

    # Show metrics

    print("✅ MST Total Cost: ",cost)
    print("⏱️ Execution Time: ",excution_time," seconds")
    print("🎥 Saving animation to: ",output_file)

    # Prepare animation

    fig, ax = plt.subplots(figsize=(6, 5))

    def init():
        ax.clear()
        nx.draw(G_full, pos, ax=ax, with_labels=True, node_color='lightgray', edge_color='lightgray')
        ax.set_title("Initial Graph")
        return ax,

    def update(frame):
        ax.clear()
        G = nx.Graph()
        step_size = step
        end_idx = min((frame + 1) * step_size, len(mst_edges))
        edges_to_show = mst_edges[:end_idx]
        G.add_weighted_edges_from(edges_to_show)
        cost1 = sum(weight for u,v, weight in edges_to_show)
        nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightgreen', edge_color='blue', width=2)
        ax.set_title(f"Step {frame+1}, showing {len(edges_to_show)} edges, current cost: {cost1}")
        return ax,

    num_steps = (len(mst_edges) + step - 1) // step  # Ceiling division
    ani = animation.FuncAnimation(
        fig, update, frames=num_steps, init_func=init, interval=interval,
        blit=False, repeat=False
    )

    ani.save(output_file, writer='ffmpeg', fps=1000//interval)

    plt.close()
    print("✅ Animation saved successfully.")

    return  HTML(ani.to_jshtml())

# **Five: Datasets**

*Dataset One: Florida*

In [ ]:
G_full=load_and_preprocess_network("/content/drive/MyDrive/analysis/eco-florida.edges")

In [ ]:
visualize_mst_animation_kruskal(G_full, output_file="eco_florida_mst_kruskal.mp4")

In [ ]:
files.download("eco_florida_mst_kruskal.mp4")

In [ ]:
visualize_mst_animation_prim_mst(G_full, output_file="eco_florida_mst_prims.mp4")

In [ ]:
files.download("eco_florida_mst_prims.mp4")

In [ ]:
visualize_mst_animation_prim_mst_using_adj(G_full, output_file="eco_florida_mst_prims_using_adj.mp4")

In [ ]:
visualize_mst_animation_boruvka_mst(G_full, output_file="eco_florida_mst_boruvka.mp4")

In [ ]:
visualize_mst_animation_reverse_delete_mst(G_full, output_file="eco_florida_mst_reverse_.mp4")

In [ ]:
visualize_mst_animation_karger_modified_mst(G_full, output_file="eco_florida_mst_karger.mp4")

*Dataset Two: Bio*

In [ ]:
G_full=load_and_preprocess_network("/content/drive/MyDrive/analysis/dubcov1.edges")

In [ ]:
visualize_mst_animation_kruskal(G_full, step=500, output_file="Dubcova_kurskal.mp4")

In [ ]:
visualize_mst_animation_prim_mst(G_full, step=500, output_file="Dubcov_primsa.mp4")

In [ ]:
visualize_mst_animation_prim_mst_using_adj(G_full, step=500, output_file="Dubcova_prims_using_adj.mp4")

In [ ]:
visualize_mst_animation_boruvka_mst(G_full, step=500, output_file="Dubcova_boruvka.mp4")

In [ ]:
visualize_mst_animation_reverse_delete_mst(G_full, step=500, output_file="Dubcova_reverse_delete.mp4")

In [ ]:
visualize_mst_animation_karger_modified_mst(G_full, step=500, output_file="Dubcova.mp4")

*Dataset Three:*

In [ ]:
G_full=load_and_preprocess_network("/content/drive/MyDrive/Analysis/G12.edges")

In [ ]:
visualize_mst_animation_kruskal(G_full,  step=10, output_file="G12_kruskal.edges.mp4")

In [ ]:
visualize_mst_animation_prim_mst(G_full, step=10, output_file="G12_prims.edges.mp4")

In [ ]:
visualize_mst_animation_prim_mst_using_adj(G_full, step=10, output_file="G12_prims_using_adj.edges.mp4")

In [ ]:
visualize_mst_animation_boruvka_mst(G_full, step=10, output_file="G12_boruvka.edges.mp4")

In [ ]:
visualize_mst_animation_reverse_delete_mst(G_full, step=10, output_file="G12_reverse_delete.edges.mp4")

In [ ]:
visualize_mst_animation_karger_modified_mst(G_full, step=10, output_file="G12_karger.edges.mp4")

*Dataset Four: GaAsH6*

In [ ]:
# GaAsH6 dataset
# kruskal MST computation on full data set
import networkx as nx, time, pandas as pd

G_full = nx.read_weighted_edgelist("/content/drive/MyDrive/analysis/GaAsH6.edges", nodetype=int)



t0 = time.time()
mst_edges = kruskal_mst_incremental(G_full)
kr_cost = (mst_edges.size(weight="weight")
           if isinstance(mst_edges, nx.Graph)
           else sum(w for _, _, w in mst_edges))
print(f"Kruskal  MST cost: {kr_cost}")
print(f"Kruskal  time (s): {round(time.time()-t0, 2)}")

In [ ]:
# prim MST computation on full data set
t0 = time.time()
mst_edges = prim_mst_incremental(G_full)
pr_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Prim     MST cost: {pr_cost}")
print(f"Prim     time (s): {round(time.time()-t0,2)}")


In [ ]:
# boruvka MST computation on full data set
t0 = time.time()
mst_edges = boruvka_mst_incremental(G_full)
bo_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Borůvka  MST cost: {bo_cost}")
print(f"Borůvka  time (s): {round(time.time()-t0,2)}")

In [ ]:
# reverse delete MST computation on full data set
t0 = time.time()
mst_edges = reverse_delete_mst_incremental(G_full)
rd_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Rev-Del  MST cost: {rd_cost}")
print(f"Rev-Del  time (s): {round(time.time()-t0,2)}")

In [ ]:
# karger MST computation on full data set
t0 = time.time()
mst_edges = karger_modified_mst(G_full, iterations=60)
kg_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Karger≈  MST cost: {kg_cost}")
print(f"Karger≈  time (s): {round(time.time()-t0,2)}")

In [ ]:
# Data sample for visualization
import networkx as nx, random

Path = "/content/drive/MyDrive/analysis/GaAsH6.edges"
G_gaas = nx.read_weighted_edgelist(Path, nodetype=int)

nodes_1k = random.sample(list(G_full.nodes()), 1000)

G_1k = G_gaas.subgraph(nodes_1k).copy()

# Keep only the largest connected component (needed for MST)
if not nx.is_connected(G_1k):
    G_1k = G_1k.subgraph(max(nx.connected_components(G_1k), key=len)).copy()

print("Sub-graph =", G_1k.number_of_nodes(), "nodes |",
                    G_1k.number_of_edges(), "edges")


nx.write_weighted_edgelist(G_1k, "/content/drive/MyDrive/analysis/GaAsH6_sample_1k.edges")

In [ ]:
G_gaas = load_and_preprocess_network("/content/drive/MyDrive/analysis/GaAsH6_sample_1k.edges")

In [ ]:
visualize_mst_animation_kruskal(G_gaas, output_file="GaAsH6_sample_mst_kruskal.mp4")

In [ ]:
files.download("GaAsH6_sample_mst_kruskal.mp4")

In [ ]:
file_name = visualize_mst_animation_prim_mst(G_gaas, output_file="GaAsH6_mst_prims.mp4")

In [ ]:
files.download(file_name)

In [ ]:
file_name=visualize_mst_animation_prim_mst_using_adj(G_gaas, output_file="GaAsH6_mst_prims_using_adj.mp4")

In [ ]:
files.download(file_name)

In [ ]:
file_name=visualize_mst_animation_boruvka_mst(G_gaas, output_file="GaAsH6_mst_boruvka.mp4")

In [ ]:
files.download(file_name)

In [ ]:
file_name=visualize_mst_animation_reverse_delete_mst(G_gaas, output_file="GaAsH6_mst_karger.mp4")

In [ ]:
files.download(file_name)

In [ ]:
file_name=visualize_mst_animation_karger_modified_mst(G_full, step=10, output_file="GaAsH6_karger.edges.mp4")

In [ ]:
files.download(file_name)

*Dataset Six: FullChip*

In [ ]:
# Full chip dataset
# kursal MST computation on full data set
import networkx as nx
import time

FULL_PATH = "/content/drive/MyDrive/analysis/FullChip.edges"
G_fullchip = nx.read_weighted_edgelist(FULL_PATH, nodetype=int, comments="%")


t0 = time.time()
mst_kr = nx.minimum_spanning_tree(G_fullchip, algorithm="kruskal")
kr_cost = mst_kr.size(weight="weight")
print("Kruskal  MST cost:", kr_cost)
print("Kruskal  time (s):", round(time.time()-t0, 2))

In [ ]:
# prim MST computation on full data set
t0 = time.time()
mst_pr = nx.minimum_spanning_tree(G_fullchip, algorithm="prim")
pr_cost = mst_pr.size(weight="weight")
print("Prim     MST cost:", pr_cost)
print("Prim     time (s):", round(time.time()-t0, 2))

In [ ]:
# boruvka MST computation on full data set
t0 = time.time()
mst_edges = boruvka_mst_incremental(G_full)
bo_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Borůvka  MST cost: {bo_cost}")
print(f"Borůvka  time (s): {round(time.time()-t0,2)}")

In [ ]:
# karger MST computation on full data set
t0 = time.time()
mst_edges = karger_modified_mst(G_full, iterations=60)
kg_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Karger≈  MST cost: {kg_cost}")
print(f"Karger≈  time (s): {round(time.time()-t0,2)}")

In [ ]:
# reverse delete MST computation on full data set
t0 = time.time()
mst_edges = reverse_delete_mst_incremental(G_full)
rd_cost   = (mst_edges.size(weight="weight")
             if isinstance(mst_edges, nx.Graph)
             else sum(w for _,_,w in mst_edges))
print(f"Rev-Del  MST cost: {rd_cost}")
print(f"Rev-Del  time (s): {round(time.time()-t0,2)}")

In [ ]:
import networkx as nx, random
import networkx as nx, random, os

RAW_PATH      = "/content/drive/MyDrive/analysis/FullChip.edges"  # original file
TARGET_EDGES  = 1_000     # edges to keep for visualisation
TARGET_NODES  = 1_000       #  node cap
SAVE_DIR      = "/content/drive/MyDrive/analysis"


#load raw graph and drop self-loops
G_full = nx.read_weighted_edgelist(RAW_PATH, nodetype=int, comments="%")
G_full.remove_edges_from(nx.selfloop_edges(G_full))

#sample random edges
sampled_edges = random.sample(
    list(G_full.edges(data=True)),
    min(TARGET_EDGES, G_full.number_of_edges())
)

G_vis = nx.Graph()
G_vis.add_weighted_edges_from(
    [(u, v, d["weight"]) for u, v, d in sampled_edges]
)

#keep the largest connected component
if not nx.is_connected(G_vis):
    G_vis = G_vis.subgraph(max(nx.connected_components(G_vis), key=len)).copy()

#trim to TARGET_NODES
if G_vis.number_of_nodes() > TARGET_NODES:
    keep = random.sample(list(G_vis.nodes()), TARGET_NODES)
    G_vis = G_vis.subgraph(keep).copy()

print(f"Sample graph → {G_vis.number_of_nodes()} nodes | {G_vis.number_of_edges()} edges")

#save sample
sample_name = f"FullChip_sample_{G_vis.number_of_nodes()}n_{G_vis.number_of_edges()}e.edges"
sample_path = os.path.join(SAVE_DIR, sample_name)
nx.write_weighted_edgelist(G_vis, sample_path)
print("Saved to:", sample_path)


G_sample = load_and_preprocess_network(sample_path)  #ready for MST functions

In [ ]:
for u, v, d in G_sample.edges(data=True):
    d['weight'] *= 10000

In [ ]:
# Kruskal's Algorithm
visualize_mst_animation_kruskal(G_full, output_file="fullchip_mst_kruskal.mp4")
files.download("fullchip_mst_kruskal.mp4")

In [ ]:
# Prim's Algorithm
visualize_mst_animation_prim_mst(G_full, output_file="fullchip_mst_prims.mp4")
files.download("fullchip_mst_prims.mp4")

In [ ]:
# Prim's Algorithm (Using Adjacency List)
file_name = visualize_mst_animation_prim_mst_using_adj(G_full, output_file="fullchip_mst_prims_using_adj.mp4")
files.download(file_name)

In [ ]:
# Borůvka's Algorithm
file_name = visualize_mst_animation_boruvka_mst(G_full, output_file="fullchip_mst_boruvka.mp4")
files.download(file_name)


In [ ]:
# Reverse Delete Algorithm
file_name = visualize_mst_animation_reverse_delete_mst(G_full, output_file="fullchip_mst_reverse.mp4")
files.download(file_name)

In [ ]:

files.download(visualize_mst_animation_karger_modified_mst(G_sample, output_file="fullchip_karger.edges.mp4"))